In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

client = QdrantClient(path="./car/")

In [ ]:
client.create_collection(
   collection_name="test",
   vectors_config={
       "title": VectorParams(
           size=1024,
           distance=Distance.COSINE,
       )}
       )

import pandas as pd
df = pd.read_csv('问题解决手册-v2 outward.csv',encoding="gbk")
df['text'] = df['报错 描述'] + ' ' + df['解决方案']
df = df.dropna()
title_des_embeds = model.encode(df['text'].tolist() ,batch_size = 64,max_length = 8192)['dense_vecs']

# NOTE: consider splitting the data into chunks to avoid hitting the server's payload size limit
# or use `upload_collection` or `upload_points` methods which handle this for you
# WARNING: uploading points one-by-one is not recommended due to requests overhead
client.upsert(
    collection_name="test",
    points=[
        PointStruct(id=i, vector={ "title_description": tit_des_embed.tolist() },
        payload={'title_description':doc},
        )
        for i, (tit_des_embed, doc) in enumerate(zip(title_des_embeds, df['text'].tolist()))
    ] 
)

In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ["HF_HOME"] = "/root/autodl-tmp/.cache/huggingface"
os.environ['LMDEPLOY_USE_MODELSCOPE'] = 'True'

#huggingface-cli download --resume-download DeepFloyd/t5-v1_1-xxl

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import pandas as pd
client = QdrantClient(path="./car/")

RuntimeError: Storage folder ./car/ is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

In [3]:

from FlagEmbedding import BGEM3FlagModel,FlagReranker

model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True) 
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) 

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [4]:
query = """I need a car with excellent sound system"""
query_embed = model.encode([query])['dense_vecs'][0]

Chunks: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


In [7]:
pd.set_option('display.max_colwidth', 500)
#Retrieve top 50
pts = client.query_points(collection_name="sales_qa", query=query_embed.tolist(),using="title_description",limit = 50).points
res = [[point.payload['title_description'],point.score] for point in pts]
res_df = pd.DataFrame(res,columns = ['title_description','score'])

#Reranking
pairs = [(query, doc) for doc in res_df.title_description.tolist()]
score = reranker.compute_score(pairs)

indices = list(range(len(score)))
indices.sort(key = lambda i:score[i],reverse=True)
reranked_chunks = [(i, pairs[i][1]) for i in indices]

Chunks:   0%|          | 0/2 [00:00<?, ?it/s]You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Chunks: 100%|██████████| 2/2 [00:02<00:00,  1.21s/it]


In [11]:
print(reranked_chunks[2][1])

2025 BMW Z4, BMW Z4 Reviews, Most Helpful Owner Reviews
#### Trending topics
* acceleration
* appearance
* handling & steering
* driving experience
* spaciousness
* ride quality
* value
* reliability & manufacturing quality
* visibility
* transmission
* seats
* comfort
* fuel efficiency
* interior
* engine
* infotainment system
* road noise
* technology
* sound system
* safety
* steering wheel
* brakes
* wheels & tires  
More
#### It's very fun to drive
5 out of 5 stars
Sanfordrich, 12/27/2019
2020 BMW Z4 sDrive M40i 2dr Convertible (3.0L 6cyl Turbo 8A)
I purchased a Misano Blue M40i (the 6 cylinder) around Aug 1. I wanted a roadster and considered a Porsche Boxster but decided the BMW was the better value. I have not regretted the decision. The BMW is a blast to drive. I think its very attractive and it even sounds good. It is very quick and responsive. A manual transmission is important to some folks but not to me. If I want to … manually shift, I can use the paddle shifters but the 